In [ ]:
!pip3 install torch tensorflow pandas numpy matplotlib tqdm

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf #needed for tb logging

In [2]:
%load_ext tensorboard

# UTILITIES

##### CONVERT NORMALISED VECTOR TO ORIGINAL FORM

In [3]:
def unNormalise(inputs, mean, std):
    """
    log to tensorboard using the writer object
    
    @params: inputs -> torch tensor of numpy.ndarray of normalised data (d1 x d2 x d3 ... x dn)
    @params: mean -> mean of along the last dimension (d1 x d2 x d3 ..... x d(n-1) x 1)
    @params: std -> std of along the last dimension (d1 x d2 x d3 ..... x d(n-1) x 1)
    
    @returns: unnormalised data (d1 x d2 x d3 .... x dn)
    """
    return inputs*std + mean

##### LOG TO TENSORBOARD

In [4]:
def log(writer, log_dict, epoch):
    """
    log to tensorboard using the writer object
    
    @params: writer -> tf.writer object
    @params: log_dict -> (name, value) pairs, log the value using the key (name) as the tag.
    @params: epoch -> the iteration number for which logging is being done.
    """
    
    with writer.as_default():
        for metric in log_dict:
            tf.summary.scalar(metric, log_dict[metric], step=epoch)

##### LOAD DATA GIVEN RELATIVE PATH FROM data_root

In [5]:
def load(rel_path):
    """
    load data from data_root/rel_path
    
    @params: rel_path -> path to a .pkl, relative to the data root global param.
    
    @returns: data -> loaded pandas df
    """
    
    path = os.path.join(data_root, rel_path)
    with open(path, "rb") as f:
        data = pd.read_pickle(f, compression=None)
    
    return data

##### MAKE DIRECTORY GIVEN THE PATH

In [6]:
def mkdir(*args, **kwargs):
    try:
        os.mkdir(*args, **kwargs)
    except Exception as e:
        print(e)

# INITIALISATIONS

##### PARAMETERS

In [7]:
BATCH_SIZE = 200000
EPOCHS = 10000
LR = 0.001
SCHEDULER_PAIENCE = 100
MODEL_ACTIVATION_FUNC = nn.Tanh

device = "cuda" if torch.cuda.is_available else "cpu"
print("using: ", device)

log_path = "./runs/tanh_train"
model_path = "./models/tanh_train"
graph_path = "./graphs"

data_root = "./datasets/"
data_dict = {"train": None, "val":None}

eps = 0.0000001

using:  cuda


##### INITIALISE DIRECTORIES AND LOGGERS

In [8]:
mkdir(log_path)
mkdir(model_path)
mkdir(graph_path)
    
train_writer = tf.summary.create_file_writer(os.path.join(log_path, "train"))
val_writer = tf.summary.create_file_writer(os.path.join(log_path, "val"))

[Errno 17] File exists: './runs/tanh_train'
[Errno 17] File exists: './models/tanh_train'
[Errno 17] File exists: './graphs'


# LOAD DATA

##### LOAD DATA TO data_dict

In [9]:
for file in os.listdir(data_root):
    if "train" in file: data_dict["train"] = load(file)
    if "test" in file: data_dict["val"] = load(file)

##### NORMALISE DATA

Calculate $train\_mean$ and $train\_std$. Use the same mean and standard deviation to normalise both train and test data.  
  
<i> POSSIBLE ERROR</i>

```---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-17-23cdf213d68d> in <module>
----> 1 train_mean = data_dict["train"].mean()
      2 train_std = data_dict["train"].std()
      3 
      4 data_dict["train"] = (data_dict["train"] - train_mean)/ train_std
      5 data_dict["test"] = (data_dict["test"] - train_mean)/train_std

AttributeError: 'NoneType' object has no attribute 'mean'
```
<i> REASON </i>  
Pandas tends to drop reference to data if not accessed for long time.   
  
<i> SOLUTION </i>  
Just run the cells to load data again. 



In [12]:
data_dict["val"]

,m,pt,phi,eta
85972,4983.729980,23798.070312,1.962157,-0.059532
38742,5435.273438,21881.867188,1.035412,0.734343
128711,5239.408691,24608.134766,-1.121055,0.828848
28751,14121.240234,203110.953125,0.324205,-2.571108
131358,3344.826660,24897.294922,0.395331,1.440069
...,...,...,...,...
133326,31842.511719,236302.125000,-2.982530,0.128011
12875,5837.973145,26775.214844,-0.797112,1.042910
111439,4847.191895,25209.994141,-2.181372,-2.281394
18479,7629.757324,37038.746094,-2.670758,-0.830225


In [10]:
train_mean = data_dict["train"].mean()
train_std = data_dict["train"].std()

data_dict["train"] = (data_dict["train"] - train_mean)/ train_std
data_dict["val"] = (data_dict["val"] - train_mean)/train_std

train_mean = train_mean.to_numpy(dtype=np.float32)
train_std = train_std.to_numpy(dtype=np.float32)

In [11]:
data_dict["train"].head(10)

,m,pt,phi,eta
132784,-0.688496,-0.607629,0.868107,0.759040
99666,-0.587358,-0.612672,-1.487534,0.117474
26629,1.051897,1.503479,-1.081401,0.773105
80473,0.788036,1.697702,-0.911068,1.813972
48229,-0.578692,-0.628716,1.619709,-0.830115
61832,-0.364437,-0.492954,-1.644013,0.033356
26867,1.190307,2.021415,1.370289,-0.926956
46232,-0.641408,-0.628934,-1.075388,-1.337238
44194,-0.593362,-0.506096,1.498136,-1.235848
59782,-0.292618,-0.545608,0.744680,-1.825007


In [12]:
data_dict["val"].head(10)

,m,pt,phi,eta
85972,-0.533282,-0.581905,1.087244,-0.071133
38742,-0.472437,-0.609328,0.573286,0.476957
128711,-0.498829,-0.570312,-0.622658,0.542203
28751,0.697978,1.984290,0.178861,-1.805121
131358,-0.754120,-0.566174,0.218307,0.964189
44010,-0.535048,-0.612530,-1.351654,-0.298414
20111,-0.579275,-0.544762,0.969280,-0.823615
137856,1.133802,1.711918,1.310452,-1.261355
87065,-0.521595,-0.625987,-0.821721,1.066632
39358,-0.090846,-0.279154,-0.322752,-0.276024


##### INITIALISE PYTORCH DATALOADERS

In [13]:
train_x = torch.tensor(data_dict["train"].to_numpy(), dtype=torch.float32)
train_y = torch.tensor(data_dict["train"].to_numpy(), dtype=torch.float32)
val_x = torch.tensor(data_dict["val"].to_numpy(), dtype=torch.float32)
val_y = torch.tensor(data_dict["val"].to_numpy(), dtype=torch.float32)

train_dataset = TensorDataset(train_x, train_y)
val_dataset = TensorDataset(val_x, val_y)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=len(val_dataset))

# MODEL

##### MODEL ARCHITECTURE

<img src="assets/Architecture%20diagram.png" alt=""/>

In [14]:
class AE(nn.Module):
    def __init__(self, input_dim, encoding_dim, activation):
        super(AE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 256),
            activation(),
            nn.Linear(256, 128),
            activation(),
            nn.Linear(128, 64),
            activation(),
            nn.Linear(64, encoding_dim),
            activation(),
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            activation(),
            nn.Linear(64, 128),
            activation(),
            nn.Linear(128, 256),
            activation(),
            nn.Linear(256, input_dim),
        )
        
    def encode(self, inputs):
        return self.encoder(inputs)
    
    def decode(self, embeddings):
        return self.decoder(embeddings)
    
    def forward(self, inputs):
        return self.decode(self.encode(inputs))
    

##### Loss function
Loss function is the simple MSE between the decoded 4D data(from 3D encodings) and the ground truth 4D data(used to generete the 3D encoding.)

In [15]:
def Loss(pred, target):
    mseLoss = nn.MSELoss()(pred,target)
    return mseLoss

##### METRICS
These metrics are tracked for the validation set, for each step of the training.  
The system for adding new metrics is modular. New metrics can be added/tracked by simply defining a function in the following form,
<pre><code>
<b>def metric_name</b>(pred: <i>torch tensor</i>, target: <i>torch tensor</i>):
    <b>return</b> metric_value: <i>1d torch tensor</i>
</code></pre>
and then adding the function name to the metrics array.  
  
All the tracked metrics appear in tensorboard with the name of the function as label.

In [16]:
def MSE_metric(pred, target):
    return nn.MSELoss()(pred, target).cpu().detach().numpy()

def MAPE_metric(pred, target):
    std = torch.from_numpy(train_std).to(device)
    mean = torch.from_numpy(train_mean).to(device)
    return  torch.abs(std*(pred - target)/(target*std+mean+eps)).mean().cpu().detach().numpy()

# TRAIN LOOP

##### INSTANTIATE MODEL, LOSS, METRICS

In [19]:
model = AE(4,3, MODEL_ACTIVATION_FUNC).to(device)
loss_func = Loss
metrics = [MAPE_metric]

##### INSTANTIATE THE OPTIMISER AND SCHEDULER
Optimiser : Adam  
Scheduler : ReduceLROnPlateau 
>Reduce Learning Rate when validation loss does not decrease for SCHEDULER_PATIENCS epochs.

In [20]:
optimiser = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, patience=SCHEDULER_PAIENCE, verbose=True, cooldown=10,factor=0.1, min_lr=1e-7)

##### TRAIN LOOP
1. The loop runs for EPOCH epochs.   
2. For each batch, after gradient decent, loss and metrics are caluclated.
3. In each epoch, the metrics and losses are calculated for both train set and val set. Actually average of metrics calculated per batch, is taken in both cases.
4. The model with the least validation loss so far is saved in model_path/best.pth and model with the latest model is stored in {model_path}/latest.pth

In [1]:
best_loss = 100000000 #the best validation loss seen so far

In [ ]:
for epoch in tqdm(range(0, EPOCHS)):
    
    # aggregators for batch metrics on train set
    batch_train_losses = []
    batch_train_metrics = {metric.__name__:[] for metric in metrics}
    
    for i, (inputs, targets) in enumerate(train_loader): # for each batch
        
        # set the initial gradients to 0
        optimiser.zero_grad()
        inputs = inputs.to(device)
        target = targets.to(device)
        
        pred = model(inputs)
        loss = loss_func(pred, target)
        loss.backward()
        
        optimiser.step()
        
        # calculate all the metrics in no_grad mode to make the calculation slightly faster
        with torch.no_grad():
            batch_train_losses.append(loss.detach().cpu().numpy())
            for metric in metrics:
                name = metric.__name__
                batch_train_metrics[name].append(metric(pred, target))
    
    # the final epoch loss and metrics for the train set
    # they are caluclated by taking mean of metrics calculated for all the batches
    train_loss = np.mean(batch_train_losses)
    train_metrics = {}
    for metric in metrics:
        name = metric.__name__
        train_metrics[name] = np.mean(batch_train_metrics[name])
    
    # calculate the val loss and val metrics in no_grad mode.
    # NOTE: since validation of the entire validation is done in 1 go,
    # we don't have any per batch aggregators (ie, no batch_val_losses)
    with torch.no_grad(): 
        for val_x, val_y in val_loader:
            val_x = val_x.to(device)
            val_y = val_y.to(device)
            
            pred = model(val_x)
            val_loss = loss_func(pred, val_y).cpu().detach().numpy()
            val_metrics = {metric.__name__:metric(pred, val_y) for metric in metrics}

    # scheduler decides on whether to change the learning rate depending
    # depending in the val_losses seen for.
    scheduler.step(val_loss)
    
    # log the train loss to tensorboard
    log(train_writer, {"loss":train_loss}, epoch)
    # log the train metrics to tensorboard
    log(train_writer, train_metrics, epoch)
    
    # log the val loss to tensorboard    
    log(val_writer, {"loss":val_loss}, epoch)
    # log the val metrics to tensorboard
    log(val_writer, val_metrics, epoch)
    
    # if val loss becomes less than the best seen so far, save the model in model_path/best.pth
    if val_loss < best_loss:
        print(f"{epoch}: val_loss improved: {best_loss}->{val_loss}")
        best_loss = val_loss
        torch.save(model.state_dict(), f"{model_path}/best.pth")
    
    # save the model in model_path/latest.pth, regardless of whether imporovement happens or not.
    if epoch%10==0: torch.save(model.state_dict(), f"{model_path}/latest.pth")

##### TENSORBOARD LOGGING

In [13]:
%tensorboard --logdir={log_path} --port=12345

# TESTING

##### LOAD BEST MODEL

In [17]:
model = AE(4,3,MODEL_ACTIVATION_FUNC)
model.to(device)
model.load_state_dict(torch.load(f"{model_path}/best.pth"))

<All keys matched successfully>

##### GENERATE THE DECODED 4D OUTPUT

In [18]:
# load val data as a torch tensor (batch size x 4)
val_data = torch.from_numpy(data_dict["val"].to_numpy(dtype=np.float32))

# pred is the predicted 4D output using the val data,
# and convert it to numpy.ndarray
pred = model(val_data.to(device)).cpu().detach().numpy()

#convert val data to a numpy.ndarray
val_data = val_data.detach().cpu().numpy()

##### UN-NORMALISE THE 4D PREDICTED OUTPUT AND 4D VALIDATION INPUT 

The model is fed val data that has been normalised (using <i> train_mean and train_std </i>).   
To get qualitative results we need to get both the predictions and val data back to the original domain.  
To get $x$ back to the original domain 
$$
x\_original = x*train\_std + train\_mean
$$

In [19]:
pred = unNormalise(pred, train_mean, train_std)
val_data = unNormalise(val_data, train_mean, train_std)

##### GRAPHING FUNCTIONS

In [20]:
def makeReconstructionBar(pred, target, name, alpha=0.5):
    """
    Creates graph for the predicted 4d output and 4d val input, such that the output is 
    layerd on top of the input, with some transparency. This way we can visually see how 
    much of a deviation there for each point.
        Savepath: graph_path/name.png
    
    target: red (background)
    pred: blue (foreground) 

    @params: pred -> predicted unnormalised values for 1 feature
    @params: target -> the ground truth unnormalised values for 1 feature
    @params: name -> name of the save file
    @params: alpha -> amount of transparency o
    """
    back = target
    front = pred
    plt.bar(np.arange(back.shape[0]), back, color="r", label="Ground Truth Value")
    plt.bar(np.arange(front.shape[0]), front, color="b", alpha=alpha, label="Reconstructed Value")
    
    plt.title(f"{name.split('.')[0]}")
    plt.xlabel("Sample points")
    plt.ylabel("Value")
    plt.legend()
    
    plt.savefig(f"{graph_path}/{name}")
    plt.close()


In [21]:
def makeMAPEScatter(pred, target, name, thresh=1):
    """
    Creates a scatter graph for the mape per point for 1 feature of the val set.
    The values greater than thresh are set equal to thresh.
        Savepath: graph_path/name.png
        
    @params: pred -> predicted unnormalised values for 1 feature
    @params: target -> the ground truth unnormalised values for 1 feature
    @params: name -> name of the save file
    @params: thresh -> the cutoff threshold.
    """
    mape = np.abs((target-pred)/(target+0.000001))
    mask = (mape>=thresh)
    mape = mape*(1-mask) + np.ones(mape.shape)*(mask)
    plt.scatter(np.arange(mape.shape[0]), mape, color="r")
    
    plt.title(f"{name.split('.')[0]}")
    plt.xlabel("Sample points")
    plt.ylabel("MAPE Value")
    plt.savefig(f"{graph_path}/{name}")
    plt.close()

In [22]:
def makeMAPEDistributionHist(pred, target, name, thresh=1):
    """
    Creates 2 graphs.
        
        1 histogram showing the number of points for which the MAPE lies in bins
        of 0.1, starting from 0 and till 1.
            Savepath: graph_path/name.png
            
        1 historgram showing the number of points for which the MAPE lies in bins
        of 0.1, starting from 0 and till 1. The y-scale is log(base 10).
            Savepath: graph_path/log_name.png
            
    @params: pred -> predicted unnormalised values for 1 feature
    @params: target -> the ground truth unnormalised values for 1 feature
    @params: name -> name of the save file
    @params: thresh -> the cutoff threshold.
    """
    mape = np.abs((target-pred)/(target+0.000001))
    mask = (mape>=thresh)
    mape = mape*(1-mask) + np.ones(mape.shape)*(mask)
    
    plt.hist(mape, log=False)
    plt.title(f"{name.split('.')[0]}")
    plt.xlabel("Bins")
    plt.ylabel("Number of points")
    plt.savefig(f"{graph_path}/{name}")
    plt.close()
    
    plt.hist(mape, log=True)
    plt.title(f"log_{name.split('.')[0]}")
    plt.xlabel("Bins")
    plt.ylabel("Number of points (log10 scale)")
    plt.savefig(f"{graph_path}/log_{name}")
    plt.close()
    

##### LOOP OVER THE FEATURES TO MAKE THE GRAPH

In [23]:
features = list(data_dict["train"].columns)
features

['m', 'pt', 'phi', 'eta']

In [24]:
for i in tqdm(range(val_data.shape[1])):
    makeMAPEScatter(pred[:,i], val_data[:,i], name=f"mape_scatter_{features[i]}.png")
    makeReconstructionBar(pred[:, i],val_data[:, i], name=f"reconstructed_bar_{features[i]}.png")
    makeMAPEDistributionHist(pred[:,i], val_data[:,i], name=f"mape_ditribution_hist_{features[i]}.png")
    

/envs/cern/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.
